# Script to Create Figure 1 - Maps of each EBUS with Trends from Model

| Region | EBUS | Temperature Cutoff | 
| ------ | ---- | :------------------: |
| California | Poleward | 11.88 | 
| California | Central | 13.07 | 
| California | Equatorward | 17.79 | 
| Humboldt | Poleward | 14.12 | 
| Humboldt | Central | 19.07 | 
| Humboldt | Equatorward | 18.27 | 
| Iberian/Canary | Poleward | 16.21 | 
| Iberian/Canary | Central | 18.95 | 
| Iberian/Canary | Equatorward | 22.57 | 
| Benguela | Poleward | 15.34 | 
| Benguela | Equatorward | 18.18 | 



In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
import datetime as dt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import math
import warnings 
warnings.simplefilter('ignore') 
import seaborn as sns
import glob

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from calendar import month_abbr
from geopy import distance

from matplotlib.ticker import StrMethodFormatter
from matplotlib import colors as c
#from matplotlib_scalebar.scalebar import ScaleBar
from datetime import datetime
import matplotlib.dates as mdates
from scalebar import scale_bar
from datetime import datetime

#specify data from Google Drive
ddir = '../../data/data_download'

### Maps

##### California

In [ ]:
#read in new climatology file
cali = xr.open_dataset(ddir+'/California_TotalClim.nc')
cali.close()

hum = xr.open_dataset(ddir+'Humboldt_TotalClim.nc')
hum.close()


beng = xr.open_dataset(ddir+'Bengula_TotalClim.nc')
beng.close()


iber = xr.open_dataset(ddir+'Iberian-Canary_TotalClim.nc')
iber.close()

#select out coordinates based on EBUS
cali_lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
cali_lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
hum_lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
hum_lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
iber_lats = np.array([[15,21.33],[21.33,30],[37, 43.39]]) 
iber_lons = np.array([[-21,-16],[-21,-9],[-14,-7]]) 
beng_lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
beng_lons = np.array([[13, 20],[10, 17],[8, 15]])

#select out sst cuttoffs 
sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (modified subregions)

#extract the minimum and maximum latitudes and longitudes from all the masks
max_lat = np.nanmax([np.nanmax(cali.analysed_sst.lat)])
min_lat = np.nanmin([np.nanmin(cali.analysed_sst.lat)])

max_lon = np.nanmax([np.nanmax(cali.analysed_sst.lon)])
min_lon = np.nanmin([np.nanmin(cali.analysed_sst.lon)])

latr = [min_lat, max_lat]
lonr = [min_lon, max_lon]

#create a region variable with a margin
margin = 1
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region_raw = np.array([[latr[0],latr[1]],[lonr[0],lonr[1]]]) 

#add state outlines
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')

#create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 300) 
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines(resolution='50m',linewidth=2,color='black', zorder = 1) 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(cfeature.BORDERS, color = 'dimgrey', linewidth = 2, zorder = 0)
ax.set_xticks([*np.arange(-180,180,5)], crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region_raw[0,0])+1,region_raw[0,1]+1,6)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
#ax.gridlines(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
ax.tick_params('both', labelsize=14, width = 2, length = 6)

lats_combined = [cali_lats, hum_lats, iber_lats, beng_lats]
lons_combined = [cali_lons, hum_lons, iber_lons, beng_lons]

cali.analysed_sst.plot(cmap = 'jet', add_colorbar=False, vmin = 11, vmax = 26)


for ds_lats, ds_lons in zip(lats_combined, lons_combined):
    for i in range(len(ds_lats)):
        rect2 = patches.Rectangle((ds_lons[i][0], ds_lats[i][0]), (ds_lons[i][1]-ds_lons[i][0]), (ds_lats[i][1]-ds_lats[i][0]), linewidth=2, edgecolor='k', facecolor='none', zorder = 3)
        ax.add_patch(rect2)
        
        try:
            cali.sel(lat = slice(ds_lats[i][0], ds_lats[i][1]), lon = slice(ds_lons[i][0], ds_lons[i][1])).analysed_sst.plot.contour(add_colorbar = False, levels=[sst_upw[i]], colors=['black'], linewidths = [4])
        except:
            pass
        
plt.text(cali_lons[0][1]+0.1, cali_lats[0][1]+0.3, "Equatorward", ha='right', fontsize = 22)
plt.text(cali_lons[1][1]+0.2, cali_lats[1][1]-0.7, "Central", ha='left', fontsize = 22)
plt.text(cali_lons[2][1]+0.2, cali_lats[2][1]-0.7, "Poleward", ha='left', fontsize = 22)

#add scalebar
scale_bar(ax, (0.1, 0.06), 2_50, text_kwargs = {'fontsize': 14})

#add north arrow
x, y, arrow_length = 0.11, 0.2, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16,
            xycoords=ax.transAxes)


plt.xlabel('Longitude', fontsize = 18)
plt.ylabel('Latitude', fontsize = 18)
plt.title('(A) California', fontsize = 24, loc = 'left')
[x.set_linewidth(2) for x in ax.spines.values()]
plt.tight_layout()
plt.savefig('../../figures/California_Subregions_annual.jpg', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()


##### Humboldt

In [ ]:
#read in new climatology file
cali = xr.open_dataset(ddir+'/California_TotalClim.nc')
cali.close()

hum = xr.open_dataset(ddir+'Humboldt_TotalClim.nc')
hum.close()

beng = xr.open_dataset(ddir+'Bengula_TotalClim.nc')
beng.close()

iber = xr.open_dataset(ddir+'Iberian-Canary_TotalClim.nc')
iber.close()

#select out coordinates based on EBUS
cali_lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
cali_lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
hum_lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
hum_lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
iber_lats = np.array([[15,21.33],[21.33,30],[37, 43.39]]) 
iber_lons = np.array([[-21,-16],[-21,-9],[-14,-7]]) 
beng_lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
beng_lons = np.array([[13, 20],[10, 17],[8, 15]])

#select out sst thresholds
sst_upw = [14.118310928344727, 18.504968643188477, 18.300222396850586] #nearshore means (modified supregions)

#extract the minimum and maximum latitudes and longitudes from all the masks
max_lat = np.nanmax([np.nanmax(hum.analysed_sst.lat)])
min_lat = np.nanmin([np.nanmin(hum.analysed_sst.lat)])

max_lon = np.nanmax([np.nanmax(hum.analysed_sst.lon)])
min_lon = np.nanmin([np.nanmin(hum.analysed_sst.lon)])

latr = [min_lat, max_lat]
lonr = [min_lon-5, max_lon]

#create a region variable with a margin
margin = 1
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region_raw = np.array([[latr[0],latr[1]],[lonr[0],lonr[1]]]) 

#create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 300) 
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines(resolution='50m',linewidth=2,color='black', zorder = 1) 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(cfeature.BORDERS, color = 'dimgrey', linewidth = 2, zorder = 2)
ax.set_xticks([*np.arange(-180,180,5)], crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0]),region[0,1]+1,10)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
#ax.gridlines(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
ax.tick_params('both', labelsize=14, width = 2, length = 6)

lats_combined = [cali_lats, hum_lats, iber_lats, beng_lats]
lons_combined = [cali_lons, hum_lons, iber_lons, beng_lons]

hum.analysed_sst.plot(cmap = 'jet', add_colorbar=False, vmin = 11, vmax = 26)

for ds_lats, ds_lons in zip(lats_combined, lons_combined):
    for i in range(len(ds_lats)):
        rect2 = patches.Rectangle((ds_lons[i][0], ds_lats[i][0]), (ds_lons[i][1]-ds_lons[i][0]), (ds_lats[i][1]-ds_lats[i][0]), linewidth=2, edgecolor='k',facecolor='none', zorder = 3)
        ax.add_patch(rect2)
        
        try:
            hum.sel(lat = slice(ds_lats[i][0], ds_lats[i][1]), lon = slice(ds_lons[i][0], ds_lons[i][1])).analysed_sst.plot.contour(add_colorbar = False, levels=[sst_upw[i]], colors=['black'], linewidths = [4])
        except:
            pass

        
plt.text(hum_lons[0][0]-0.2, hum_lats[0][1]-1.1, "Poleward", ha='right', fontsize = 22)
plt.text(hum_lons[1][0]-0.3, hum_lats[1][1]-1.6, "Central", ha='right', fontsize = 22)
plt.text(hum_lons[2][0]+2.1, hum_lats[2][1]+0.4, "Equatorward", ha='right', fontsize = 22)

#add scalebar
scale_bar(ax, (0.1, 0.04), 2_50, text_kwargs = {'fontsize': 14})

#add north arrow
x, y, arrow_length = 0.11, 0.2, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16,
            xycoords=ax.transAxes)

plt.xlabel('Longitude', fontsize = 18)
plt.ylabel('Latitude', fontsize = 18)
plt.title('(C) Humboldt', fontsize = 24, loc = 'left')
[x.set_linewidth(2) for x in ax.spines.values()]

plt.tight_layout()
plt.savefig('../../figures/Humboldt_Subregions_annual.jpg', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()


##### Iberian-Canary

In [ ]:
#read in new climatology file
cali = xr.open_dataset(ddir+'/California_TotalClim.nc')
cali.close()

hum = xr.open_dataset(ddir+'Humboldt_TotalClim.nc')
hum.close()

beng = xr.open_dataset(ddir+'Bengula_TotalClim.nc')
beng.close()

iber = xr.open_dataset(ddir+'Iberian-Canary_TotalClim.nc')
iber.close()

#select out coordinates based on EBUS
cali_lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
cali_lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
hum_lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
hum_lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
iber_lats = np.array([[15,21.33],[21.33,33],[37, 43.39]]) 
iber_lons = np.array([[-21,-16],[-21,-8.75],[-14,-7]]) 
beng_lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
beng_lons = np.array([[13, 20],[10, 17],[8, 15]])

#select out sst cuttof
sst_upw = [22.11833381652832, 18.946455001831055, 16.212278366088867] #nearshore means (modified subregions with central Iberian changed)

#extract the minimum and maximum latitudes and longitudes from all the masks
max_lat = np.nanmax([np.nanmax(iber.analysed_sst.lat)])
min_lat = np.nanmin([np.nanmin(iber.analysed_sst.lat)])

max_lon = np.nanmax([np.nanmax(iber.analysed_sst.lon)])
min_lon = np.nanmin([np.nanmin(iber.analysed_sst.lon)])

latr = [min_lat, max_lat]
lonr = [min_lon, max_lon]

#create a region variable with a margin
margin = 1
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin-1,lonr[1]+margin]]) 
region_raw = np.array([[latr[0],latr[1]],[lonr[0],lonr[1]]]) 

#create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 300) 
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines(resolution='50m',linewidth=2,color='black', zorder = 1) 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(cfeature.BORDERS, color = 'dimgrey', linewidth = 2, zorder = 0)
ax.set_xticks(np.round([*np.arange(region_raw[1,0]-2,region_raw[1,1]+1,6)],0), crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0])+3,region[0,1]+1,10)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
#ax.gridlines(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
ax.tick_params('both', labelsize=14, width = 2, length = 6)

lats_combined = [cali_lats, hum_lats, iber_lats, beng_lats]
lons_combined = [cali_lons, hum_lons, iber_lons, beng_lons]

iber.analysed_sst.plot(cmap = 'jet', add_colorbar=False, vmin = 11, vmax = 26)

for ds_lats, ds_lons in zip(lats_combined, lons_combined):
    for i in range(len(ds_lats)):
        rect2 = patches.Rectangle((ds_lons[i][0], ds_lats[i][0]), (ds_lons[i][1]-ds_lons[i][0]), (ds_lats[i][1]-ds_lats[i][0]), linewidth=2, edgecolor='k', facecolor='none', zorder = 3)
        ax.add_patch(rect2)
        
        try:
            iber.sel(lat = slice(ds_lats[i][0], ds_lats[i][1]), lon = slice(ds_lons[i][0], ds_lons[i][1])).analysed_sst.plot.contour(add_colorbar = False, levels=[sst_upw[i]], colors=['black'], linewidths = [4])
        except:
            pass
        
plt.text(iber_lons[0][0]+3.5, iber_lats[0][1]-7.9, "Equatorward", ha='right', fontsize = 22)
plt.text(iber_lons[1][0]-0.2, iber_lats[1][1]-1.0, "Central", ha='right', fontsize = 22)
plt.text(iber_lons[2][0]-0.2, iber_lats[2][1]-0.9, "Poleward", ha='right', fontsize = 22)

#add scalebar
scale_bar(ax, (0.1, 0.93), 2_50, text_kwargs = {'fontsize': 14})

#add north arrow
x, y, arrow_length = 0.11, 0.905, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16,
            xycoords=ax.transAxes)

plt.xlabel('Longitude', fontsize = 18)
plt.ylabel('Latitude', fontsize = 18)
plt.title('(B) Iberian/Canary', fontsize = 24, loc = 'left')
[x.set_linewidth(2) for x in ax.spines.values()]
plt.tight_layout()
plt.savefig('../../figures/Iberian-Canary_Subregions_annual.jpg', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()


##### Benguela

In [ ]:
#read in new climatology file
cali = xr.open_dataset(ddir+'/California_TotalClim.nc')
cali.close()

hum = xr.open_dataset(ddir+'Humboldt_TotalClim.nc')
hum.close()

beng = xr.open_dataset(ddir+'Bengula_TotalClim.nc')
beng.close()

iber = xr.open_dataset(ddir+'Iberian-Canary_TotalClim.nc')
iber.close()

#select out coordinates based on EBUS
cali_lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
cali_lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
hum_lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
hum_lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
iber_lats = np.array([[15,21.33],[21.33,30],[37, 43.39]]) 
iber_lons = np.array([[-21,-16],[-21,-9],[-14,-7]]) 
beng_lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
beng_lons = np.array([[13, 20],[10, 17],[8, 15]])

#select out sst cutoff
sst_upw = [15.6278657913208, 15.074623107910156, 18.181407928466797] #nearshore means (modified subregions)

#extract the minimum and maximum latitudes and longitudes from all the masks
max_lat = np.nanmax([np.nanmax(beng.analysed_sst.lat)])
min_lat = np.nanmin([np.nanmin(beng.analysed_sst.lat)])

max_lon = np.nanmax([np.nanmax(beng.analysed_sst.lon)])
min_lon = np.nanmin([np.nanmin(beng.analysed_sst.lon)])

latr = [min_lat, max_lat]
lonr = [min_lon, max_lon]

#create a region variable with a margin
margin = 1
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region_raw = np.array([[latr[0],latr[1]],[lonr[0],lonr[1]]]) 

#create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 300) 
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines(resolution='50m',linewidth=2,color='black', zorder = 1) 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(cfeature.BORDERS, color = 'dimgrey', linewidth = 2, zorder = 0)
ax.set_xticks(np.round([*np.arange(region_raw[1,0],region_raw[1,1]+1,5)],0), crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0])+3,region[0,1]+1,10)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
#ax.gridlines(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
ax.tick_params('both', labelsize=14, width = 2, length = 6)

lats_combined = [cali_lats, hum_lats, iber_lats, beng_lats]
lons_combined = [cali_lons, hum_lons, iber_lons, beng_lons]

fg = beng.analysed_sst.plot(cmap = 'jet', vmin = 11, vmax = 26, add_colorbar=False)
cb = plt.colorbar(fg, pad = 0.02)      
cb.set_label(label = 'Sea Surface Temperature (SST, °C)', size = 16, labelpad = 10)
cb.ax.tick_params(labelsize = 14)


for ds_lats, ds_lons in zip(lats_combined, lons_combined):
    for i in range(len(ds_lats)):
        rect2 = patches.Rectangle((ds_lons[i][0], ds_lats[i][0]), (ds_lons[i][1]-ds_lons[i][0]), (ds_lats[i][1]-ds_lats[i][0]), linewidth=2, edgecolor='k', facecolor='none', zorder = 3)
        ax.add_patch(rect2)
        
        try:
            beng.sel(lat = slice(ds_lats[i][0], ds_lats[i][1]), lon = slice(ds_lons[i][0], ds_lons[i][1])).analysed_sst.plot.contour(add_colorbar = False, levels=[sst_upw[i]], colors=['black'], linewidths = [4])
        except:
            pass
        
plt.text(beng_lons[0][0]-0.3, beng_lats[0][1]-1.2, "Poleward", ha='right', fontsize = 22)
plt.text(beng_lons[1][0]-0.3, beng_lats[1][1]-1.2, "Central", ha='right', fontsize = 22)
plt.text(beng_lons[2][0]+4, beng_lats[2][1]+0.3, "Equatorward", ha='right', fontsize = 22)

#add scalebar
scale_bar(ax, (0.1, 0.04), 2_50, text_kwargs = {'fontsize': 14})

#add north arrow
x, y, arrow_length = 0.119, 0.175, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16,
            xycoords=ax.transAxes)

plt.xlabel('Longitude', fontsize = 18)
plt.ylabel('Latitude', fontsize = 18)
plt.title('(D) Benguela', fontsize = 24, loc = 'left')
[x.set_linewidth(2) for x in ax.spines.values()]
plt.tight_layout()
plt.savefig('../../figures/Benguela_Subregions_annual.jpg', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()


### Trend Plots

##### Open Trends

In [ ]:
#open file
trends = pd.read_csv('../../data/EBUS_glm_FullModel_Trends_Modified-Subregions.csv', index_col = 0)
trends = trends.dropna()

#create a column for significant trends
trends['sig'] = ''
#if the monte carlo p value is less than 0.025 (this is one tailed test)
trends.loc[trends['mc.p.value'] <= 0.025, 'sig'] = '*'

#add a column for subregion numbers and color
trends['Subregion_num'] = np.nan
trends['Color'] = 'dodgerblue'
        
#fill in the subregion numbers and colors for significant trends
trends.loc[(trends['EBUS'] == 'California') & (trends['Subregion'] == 'Equatorward'), ['Subregion_num']] = 0.25
trends.loc[(trends['EBUS'] == 'California') & (trends['Subregion'] == 'Central'), ['Subregion_num']] = 1
trends.loc[(trends['EBUS'] == 'California') & (trends['Subregion'] == 'Poleward'), ['Subregion_num']] = 1.78

trends.loc[(trends['EBUS'] == 'Iberian/Canary') & (trends['Subregion'] == 'Equatorward'), ['Subregion_num']] = -0.05
trends.loc[(trends['EBUS'] == 'Iberian/Canary') & (trends['Subregion'] == 'Central'), ['Subregion_num']] = 0.75
trends.loc[(trends['EBUS'] == 'Iberian/Canary') & (trends['Subregion'] == 'Poleward'), ['Subregion_num']] = 1.935

trends.loc[(trends['EBUS'] == 'Humboldt') & (trends['Subregion'] == 'Equatorward'), ['Subregion_num']] = 2.05
trends.loc[(trends['EBUS'] == 'Humboldt') & (trends['Subregion'] == 'Central'), ['Subregion_num']] = 1.33
trends.loc[(trends['EBUS'] == 'Humboldt') & (trends['Subregion'] == 'Poleward'), ['Subregion_num']] = 0.3

trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Equatorward'), ['Subregion_num']] = 1
trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Central'), ['Subregion_num']] = .5
trends.loc[(trends['EBUS'] == 'Benguela') & (trends['Subregion'] == 'Poleward'), ['Subregion_num']] = 0

trends.loc[trends['sig'] == '*', 'Color'] = 'red'

#replace the - with / in Iberian/Canary
trends['EBUS'] = trends['EBUS'].str.replace('-','/')
trends

In [ ]:
#loop through all of the ebus
for i, ebus in zip([*range(4)], ['California', 'Iberian/Canary', 'Humboldt', 'Benguela']):
    
    fig, axs = plt.subplots(1, 1, figsize=(5, 10), dpi = 300)

    #filter the data
    temp = trends[trends['EBUS'] == ebus]
    
    #calculate the confidence intervals
    ytop = temp['upper.CL.gls']-temp['Year.trend.gls']
    ybot = temp['Year.trend.gls']-temp['lower.CL.gls']
    
    #plot the data
    axs.scatter(temp['Year.trend.gls'], temp['Subregion_num'], c = temp['Color'], edgecolor = 'k', marker = 'D', s = 250, zorder = 2, linewidth = 2)
    axs.scatter(temp['Year.trend.lm'], temp['Subregion_num'], c = temp['Color'], edgecolor = 'k', marker = 'o', s = 250, zorder = 2, linewidth = 2)
    
    #add the error bars
    axs.errorbar(temp['Year.trend.gls'], temp['Subregion_num'], xerr =(ybot, ytop), fmt='.k', 
                    c = 'k', capsize=10, zorder = 1, linewidth = 2, markeredgewidth=2)
        
    #add a line for 0
    axs.axvline(c = 'k', linewidth = 3, alpha=0.5, linestyle = 'dashed', zorder = 0)
    #add a title for each EBUS
    axs.set_title(ebus, fontsize = 18)
    
    #if the ebus is California
    if ebus == 'California':
        #add space around the x-axis and add three subregion labels
        axs.set_yticks([-0.5, 0.25, 1.0, 1.78, 2.5])
        axs.set_yticklabels(['', 'Equatorward', 'Central', 'Poleward', ''], fontsize = 14)

    #if the ebus is Iberian/Canary
    elif ebus == 'Iberian/Canary': 
        #add space around the x-axis and add three subregion labels
        axs.set_yticks([-0.5, -0.05, 0.75, 1.935, 2.5])
        axs.set_yticklabels(['', 'Equatorward', 'Central', 'Poleward', ''], fontsize = 14)

    #if the ebus is Humboldt
    elif ebus == 'Humboldt': 
        #add space around the x-axis and add three subregion labels
        axs.set_yticks([-0.5, 0.3, 1.33, 2.05, 2.5])
        axs.set_yticklabels(['', 'Poleward', 'Central', 'Equatorward', ''], fontsize = 14)
        
    #if the ebus is Benguela
    else: 
        #add space around the x-axis and add the two subregion labels
        axs.set_yticks([-0.5, 0, 0.5, 1, 1.5])
        axs.set_yticklabels(['', 'Poleward', 'Central', 'Equatorward', ''], fontsize = 14) 
        
    #add a y-axis label
    axs.set_xlabel('Trend (percent area / year)', fontsize = 16)

    #set the y-axis values so all the ebus are the same
    axs.set_xticks([-2, -1, 0, 1, 2])
    axs.set_xticklabels([-2, -1, 0, 1, 2], fontsize = 14)
    #axs.grid(True, alpha=0.5, zorder=0)
    axs.tick_params('both', labelsize=14, width = 2, length = 6)

    
    #add the markers for the significance 
    for index, txt in enumerate(temp['sig']):
        if ebus == 'Benguela':
            axs.annotate(txt, (1.5, temp['Subregion_num'].iloc[index]-0.052), fontsize = 32, weight='bold')
        else:
            axs.annotate(txt, (1.5, temp['Subregion_num'].iloc[index]-0.077), fontsize = 32, weight='bold')
            
    [x.set_linewidth(2) for x in axs.spines.values()]
    plt.tight_layout()
    if ebus == 'Iberian/Canary':
        plt.savefig('../../figures/Iberian_Canary_Trend.jpg', facecolor='white', bbox_inches = 'tight', dpi = 300)
    else:
        plt.savefig('../../figures/'+ebus+'_Trend.jpg', facecolor='white', bbox_inches = 'tight', dpi = 300)

            
    plt.show()